In [1]:
#import all necessary packages
import pandas as pd
from tqdm import tqdm
import spacy
from spacy.tokens import DocBin
import re
import os
from spacy import displacy
from sklearn import metrics
from sklearn.metrics import plot_confusion_matrix

from spacy.util import filter_spans

In [2]:
FOODKEEPER_PATH = "datasets/FoodKeeper-Data.xls"
TRAINING_DATA_PATH = "datasets/response.txt"
MODEL_PATH = "output/model-last"
TEST_DATA_PATH = "datasets/test_data.csv"

pd.options.mode.chained_assignment = None

#creates a blank pipeline of a given language class - english
#function is twin of spacy.load()
nlp = spacy.blank("en")

#Read a comma-separated values (csv) file into DataFrame
#Also supports optionally iterating or breaking the file into chunks
food_data = pd.read_excel(FOODKEEPER_PATH, sheet_name = "Product")
training_data = pd.read_csv(TRAINING_DATA_PATH,index_col = False, header = None)
test_data = pd.read_csv(TEST_DATA_PATH)
print(training_data)

#loop through and count the specific entities
keywords = [] #'chicken', 'milk', 'butter', 'cheese'
sampleData = []

                                                    0
0                     a pizza with cheese and spinach
1                          a pie with a slice missing
2                     a sandwich with meat and cheese
3                       a cheeseburger on a newspaper
4                             a white cake on a plate
5        a hand holding a paper plate of french fries
6              a large burger with lettuce and tomato
7                             a hand holding a cookie
8           a slice of cake with whipped cream on top
9               a stack of pancakes with fruit on top
10                  a plate of seafood and vegetables
11              a person holding a bunch of cucumbers
12                       slices of oranges on a plate
13  a group of cupcakes with frosting and candy on...
14                                a group of tomatoes
15                            a close-up of some eggs
16             a chocolate donut with sprinkles on it
17                          

In [3]:
# makes a list of all the keywords from FoodKeeper database
def foodKeeperInfo():              
    keywords = []
    for word in food_data['Name']:
        word = word.replace(" or ", " ")
        word = re.sub('[/,]', ' ', word)
        word = word.lstrip()
        word = word.rstrip()

        if word.lower() not in keywords: 
            keywords.append(word.lower())

    #print("Total foodkeeper food names: " + str(len(keywords)))        
    #for element in sorted(keywords):
        #print(element)
        
    return keywords

foodKeeperKeywordsTest = foodKeeperInfo()
foodKeeperKeywordsTest

['butter',
 'buttermilk',
 'cheese',
 'coffee creamer',
 'cottage cheese',
 'cream cheese',
 'cream',
 'dips',
 'egg substitutes',
 'eggnog',
 'eggs',
 'egg dishes',
 'kefir',
 'margarine',
 'milk',
 'pudding',
 'sour cream',
 'whipped cream',
 'whipped topping',
 'yogurt',
 'beef',
 'lamb',
 'veal',
 'pork',
 'goat',
 'venison',
 'variety meats',
 'bacon',
 'corned beef',
 'ham',
 'hot dogs',
 'sausage',
 'stuffed  raw pork chops',
 'raw kabobs with vegetables',
 'jerky',
 'meat products',
 'retort pouches boxes',
 'chicken',
 'turkey',
 'ground turkey chicken',
 'chicken parts',
 'turkey parts',
 'duckling',
 'goose',
 'pheasant',
 'quail',
 'capon',
 'cornish hens',
 'giblets',
 'stuffed  raw chicken breasts',
 'turducken',
 'chicken nuggets  patties',
 'cooked poultry dishes',
 'fried chicken',
 'poultry pieces',
 'rotisserie chicken',
 'canned chicken',
 'lean fish',
 'fatty fish',
 'caviar',
 'cooked fish',
 'surimi seafood',
 'scallops',
 'shrimp  crayfish',
 'squid',
 'shucked 

In [4]:
#gets the functions from foodKeeperFunctions file
%run foodKeeperFunctionsInsta.ipynb

def findNewKeywords(post, keywords):
    foodkeeperKeys = foodKeeperInfo()
    x = post.split()
    word = ""
    i = 0
    while i < len(x):
    #for i in range(len(x)):
        z = 1
        if x[i] in foodkeeperKeys:
            word = x[i]
        try:
            foundBiWord = x[i] + " " + x[i+1]
            if foundBiWord in foodkeeperKeys: #keywords
                word = foundBiWord
                z = 2
        except:
            pass
        
        try:
            foundTriWord = x[i] + " " + x[i+1] + " " + x[i+2]
            if foundTriWord in foodkeeperKeys: #keywords:
                word = foundTriWord
                z = 3
        except:
            pass
        i += z
        
        if word not in keywords and word != "":
            keywords.append(word)
    return keywords

def ent_recognize(text):
    doc = nlp(text)
    displacy.render(doc,style = "ent")


In [5]:
test = []
keywordRanker = {}
myInstaPost = []
keywords = ['noodles', 'shrimp', 'soup']
x = convertToTrainingFormat(training_data[0][20], keywords)
myInstaPost.append(x)
for elem in myInstaPost:
    print(elem)
    findNewKeywords(elem[0], test)
#print(test)

()


IndexError: tuple index out of range

In [ ]:
def getCommonVerbs(data):
    import en_core_web_sm
    nlp2 = spacy.load("en_core_web_sm")
    count = 0
    myVerbs = {}
    for i in range(len(data[0])):
        doc = nlp2(data[0][i])
        for token in doc:
            if token.pos_ == "VERB":
                if token.text in myVerbs:
                    myVerbs[token.text] = myVerbs[token.text] + 1
                else:
                    if token.text not in nlp2.Defaults.stop_words:
                        myVerbs[token.text] = 1
        

    topVerbs = dict(sorted(myVerbs.items(), key = lambda item: item[1], reverse=True)[:10])
    return [key for key in topVerbs]

In [6]:
def preProcess(tweet):
    #Converts a tweet to lowercase, replaces anyusername w/ <USERNAME> and URLS with <URL>
    tweet = tweet.lower()
    tweet = re.sub('@[a-zA-z0-9]*', '', tweet)              # <USERNAME>
    tweet = re.sub('http[a-zA-z0-9./:]*', '', tweet)       # <URL>
    tweet = re.sub('[.,-]*', '', tweet)
    tweet = re.sub('&amp;', 'and', tweet)
    
    return tweet

noEntity= []

keywordRanker = {}

def trainModel(data):
    #Initialize all the variables
    keywords = [] #foodKeeperInfo()
    oldKeywords = []
    newKeywords = []
    
      
    
    commonVerbs = getCommonVerbs(data) 
    print(commonVerbs)
    print("Common Verbs gathered...", '\n')
    
    #entityCheckCount controls how many entities are required to 
    #add a Tweet to be trained
    
    entityCheckCount = 3
    
    counter = 0
    trainingLoop = True
    
    while trainingLoop:
        counterText = "~~~~~~~~~~~~~~~~~"+str(counter)+"~~~~~~~~~~~~~~~~~"
        print(counterText)
        
        nlp = spacy.blank("en") # load a new spacy model
        try:
            model = spacy.load(MODEL_PATH)
            print('Model loaded...')
        except:
            print('No model...')
        
        db = DocBin() # create a DocBin object
        
        myTweets = []
        
        #Loop through all the tweets
        #This loop is necessary to get the most common keywords 
        #in the convertToTrainingFormat function
        
        for i in range(len(data[0])): #len(data[0])
            #useless if?
#             if i % 500 == 0:
#                 print(i)

            if counter == 0:
                x = convertToTrainingFormat(preProcess(data[0][i]), keywords)
                
            #If counter is 1 then there is no model to check so 
            #a word count is performed
            elif counter == 1:
                x = convertToTrainingFormat(preProcess(data[0][i]), keywords)
                if x!= ():
                    if len(x[1]['entities']) > entityCheckCount:
                        #print("Found tweet", x[0])
                        myTweets.append(x)  
            else:
                #Convert each tweet into spacy training format
                x = convertToTrainingFormat(preProcess(data[0][i]), keywords)
                checkPassed = False
                if x != ():
                    #Check the ranking of the tweet
                    if rankTweet(x[0], model) > entityCheckCount:
                        checkTweet = x[0].split()
                        
                        #Check to see if tweet has one of the common verbs
                        for word in checkTweet:
                            if word in commonVerbs:
                                checkPassed = True
                                
                        if True: #checkPassed:
                            #print("Checking rank...")
                            myTweets.append(x)
        
                        
#         Initialize the keywords
       
        
        if counter == 0:  
            # Set keywords to be all keywords found in foodkeeper
#            keywords = foodKeeperInfo()
            sortedKeywords =  sorted(keywordRanker, key=keywordRanker.get, reverse=True)

            for i in range(15): #sortedKeywords
                keywords.append(sortedKeywords[i])
            #print(sortedKeywords[i], keywordRanker[sortedKeywords[i]])
            

        elif counter > 0:
            for text, annot in tqdm(myTweets): # data in previous format
                doc = nlp.make_doc(text) # create doc object from text
                ents = []
                for start, end, label in annot["entities"]: # add character indexes
                    span = doc.char_span(start, end, label=label, alignment_mode="contract")
                    
                    if span is None:
                        print("Skipping entity")
                    elif ents == []: ents.append(span) 
                    else:
                        #Check to see if any entities are overlapping i.e rice and rice cakes
                        ents.append(span)
                        
                        
#                         for ent in ents:
#                             if ent is not None:
#                                 entLength = ent.end - ent.start
#                                 if span.start == ent.start or span.end == ent.end:
#                                     if entLength > (span.end - span.start):
#                                         continue
#                                     else:
#                                         ents.remove(ent)
#                                         ents.append(span)
#                                 else:
#                                     ents.append(span)
                    #if span not in ents: ents.append(span)
                            
                                #print(span.start, span.end, ents)
                    #put into for loop
#                 foundEnts = []
#                 newEnts = []
#                 for ent in ents:
#                     if ent.text not in foundEnts:
#                         foundEnts.append(ent.text)
#                         newEnts.append(ent)
                        
                newEnts = filter_spans(ents)              
                #try:
                doc.ents = newEnts # label the text with the ents
                    #print(doc)
                db.add(doc)
                #except:
                    #print("Error 10: ", doc)

            db.to_disk("./train.spacy") # save the docbin object
                
            #If problems are occuring with the models not appearing
            #ensure that the command is valid, specifically python is the correct
            #PATH variable name on your machine
            #--paths.train should be where the docbin object is saved
            stream = os.popen('python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./train.spacy')
            print(stream.read())
            print("Total keywords: ", str(len(keywords)))
            print("List of Keywords:\n\n",keywords,"\n\n")
            oldKeywords = len(keywords)
            
            
            #Look for new keywords
            for element in myTweets:
                keywords = findNewKeywords(element[0], keywords)
                
            
            #No new keywords are found
            if (oldKeywords == len(keywords)) and counter > 1 and entityCheckCount != 1:
                entityCheckCount -= 1
                print("Decreasing entityCheckCount variable by 1")
                print("entityCheckCount = ", entityCheckCount)
                

            #New keywords are found and entity rank check == 1
            elif (oldKeywords == len(keywords)) and counter > 1 and entityCheckCount == 1:
                trainingLoop = False
            
            eval_model()
            
        

        #for element in myTweets:
            #findNewKeywords(element[0], keywords)

        print("Total keywords: ", str(len(keywords)))
        print("Total Tweets: ", str(len(myTweets)))
        print("List of Keywords:\n\n",keywords,"\n\n")
        counter += 1
        
    print('Training Done...')
        
        
        

def information(data):
    myData = {}
    totalEnt=0
    
    for i in range(len(data[0])):
        doc = nlp(preProcess(data[0][i]))
        if len(doc.ents) == 0:
            noEntity.append(preProcess(data[0][i]))
            
        #print(len(doc.ents))
        if(len(doc.ents) == 4):
            print(doc)
            
        for entity in doc.ents: 
        #print(entity.label_)
            totalEnt+=1
            if(entity.label_ == 'FOOD'):
                if entity.text in myData:
                    myData[entity.text] += 1
                else:
                    myData[entity.text] = 1
                    
    print("Number of entities found: " + str(len(myData)))
    print(totalEnt)
    for i in sorted(myData, key = myData.get):
        print("Entity: " + i, "Count: " + str(myData[i]), "Density: " + str(format(myData[i]/totalEnt, '.2f')), end = "\n")
    
    
    return myData




for i in range(len(test_data['tweet'])):
    test_data['tweet'][i] = preProcess(test_data['tweet'][i])

y = test_data['food'].tolist()
# print(test_data)
# print(nlp.pipe_names)
    
def ent_recognize(text):
    doc = nlp(text)
    displacy.render(doc,style = "ent")